## mini func, check if a query is on topic or off topic , first

In [1]:
from transformers import pipeline, BertTokenizer, BertModel
import torch

# Load Hugging Face pipeline for zero-shot classification
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def check_if_on_topic(query, topics):
    
    classification = classifier(query, topics, multi_label=True)
    return classification



queries = [
    "I'm looking for good wings near me",
    "What is a good chicken wings place near me?",
    "Are there any good pasta places around here?",
    "What is a good italian place nearby?",
    "What is a decent burger spot here?",
    "Are there some decent fast food spots nearby?",
    "What is a good noodle place in town?",
]
topics = [
    "this is about food",
    "food",
    "this is about a restaurant",
    "restaurants",
    "food places",
    "this is about food places",
]
for query in queries:
    classification = check_if_on_topic(query, topics)
    print("")
    print("query", query)
    print(classification)



query I'm looking for good wings near me
{'sequence': "I'm looking for good wings near me", 'labels': ['food places', 'this is about food places', 'this is about food', 'this is about a restaurant', 'food', 'restaurants'], 'scores': [0.9868150949478149, 0.984097421169281, 0.9606814384460449, 0.8281988501548767, 0.7463715672492981, 0.5991970300674438]}

query What is a good chicken wings place near me?
{'sequence': 'What is a good chicken wings place near me?', 'labels': ['food', 'food places', 'this is about food', 'this is about food places', 'this is about a restaurant', 'restaurants'], 'scores': [0.9929962158203125, 0.9925386309623718, 0.991735577583313, 0.9857849478721619, 0.9375878572463989, 0.9168540835380554]}

query Are there any good pasta places around here?
{'sequence': 'Are there any good pasta places around here?', 'labels': ['food places', 'this is about food', 'food', 'this is about food places', 'this is about a restaurant', 'restaurants'], 'scores': [0.988788068294525

Ok wow, kind of like I recall reading about Zero Shot learning recently, using BART , seems the one-word MNLI use is not so great, also for this super limited example here.


Let's mix in also some counter cases too? 

In [5]:
sum(classification["scores"])

5.554864883422852

In [6]:


queries = [
    ["I'm looking for good wings near me", 1],
    ["What is a good chicken wings place near me?", 1],
    ["Are there any good pasta places around here?", 1],
    ["What is a good italian place nearby?", 1],
    ["What is a decent burger spot here?", 1],
    ["Are there some decent fast food spots nearby?", 1],
    ["What is a good noodle place in town?", 1],
    ["I am looking for medical help", 0],
    ["I want to learn about painting", 0],
    ["Where is a good place to do my taxes?", 0],
    ["I want to go somewhere and play soccer.", 0],
    ["Where is a good public school for my kids?", 0],
    ["What are good movies to see these days?", 0],
    ["Any good bowling alleys nearby?", 0],
    ["What is a good activity as a tourist to do some sight seeing maybe?", 0],
    ["I am looking for a good museum for art history", 0],
    ["I am hungry where can I go? Hungry for japanese food.", 1],
    ["I want to eat some yummy tacoes with a small group of people. ", 1],
    ["Ok any chicken wings places in the area?", 1],
    ["What is a good meat place like for smoked meat?", 1],
    ["I am in the mood for some ramen, any nearby?", 1],
]
topics = [
    "this is about food",
    "food",
    "this is about a restaurant",
    "restaurants",
    "food places",
    "this is about food places",
]

scores = []

len_food_topics = len(topics)
for query, food_true in queries:
    classification = check_if_on_topic(query, topics)

    food_pred = sum(classification["scores"]) / len_food_topics
    scores.append([query, food_pred, food_true, abs(food_true - food_pred)])

sorted(scores, key=lambda x:x[-1])




[['I am looking for medical help',
  8.088221087139875e-05,
  0,
  8.088221087139875e-05],
 ['I am looking for a good museum for art history',
  9.62637289679454e-05,
  0,
  9.62637289679454e-05],
 ['I want to learn about painting',
  0.00011275537811646548,
  0,
  0.00011275537811646548],
 ['I want to go somewhere and play soccer.',
  0.00011537025845124542,
  0,
  0.00011537025845124542],
 ['Where is a good place to do my taxes?',
  0.00014225074361699322,
  0,
  0.00014225074361699322],
 ['Any good bowling alleys nearby?',
  0.0001614166055029879,
  0,
  0.0001614166055029879],
 ['Where is a good public school for my kids?',
  0.00023048519263587272,
  0,
  0.00023048519263587272],
 ['What is a good activity as a tourist to do some sight seeing maybe?',
  0.0002453784157599633,
  0,
  0.0002453784157599633],
 ['What are good movies to see these days?',
  0.0025914414545695763,
  0,
  0.0025914414545695763],
 ['What is a good chicken wings place near me?',
  0.9695828855037689,
  1,


In [7]:
for row in sorted(scores, key=lambda x:x[-1]):
    print(f"{row[0]}: delta: {row[-1]:.2f} (actual: {row[1]:.2f}, true: {row[2]:.2f})")


I am looking for medical help: delta: 0.00 (actual: 0.00, true: 0.00)
I am looking for a good museum for art history: delta: 0.00 (actual: 0.00, true: 0.00)
I want to learn about painting: delta: 0.00 (actual: 0.00, true: 0.00)
I want to go somewhere and play soccer.: delta: 0.00 (actual: 0.00, true: 0.00)
Where is a good place to do my taxes?: delta: 0.00 (actual: 0.00, true: 0.00)
Any good bowling alleys nearby?: delta: 0.00 (actual: 0.00, true: 0.00)
Where is a good public school for my kids?: delta: 0.00 (actual: 0.00, true: 0.00)
What is a good activity as a tourist to do some sight seeing maybe?: delta: 0.00 (actual: 0.00, true: 0.00)
What are good movies to see these days?: delta: 0.00 (actual: 0.00, true: 0.00)
What is a good chicken wings place near me?: delta: 0.03 (actual: 0.97, true: 1.00)
What is a good italian place nearby?: delta: 0.03 (actual: 0.97, true: 1.00)
What is a good noodle place in town?: delta: 0.07 (actual: 0.93, true: 1.00)
What is a decent burger spot here

## ok that is not that bad 
maybe should look at more data, but that's looking pretty decent for a first stab with how well BART is doing here 